In [1]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr


In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()


## Part 1


Read the case.csv file from the 311 call data into a Spark DataFrame.


In [3]:
df = spark.read.csv('case.csv', header=True)

In [4]:
df.columns

['case_id',
 'case_opened_date',
 'case_closed_date',
 'SLA_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'dept_division',
 'service_request_type',
 'SLA_days',
 'case_status',
 'source_id',
 'request_address',
 'council_district']

How old is the latest (in terms of days past SLA) currently open issue?

In [5]:
(df
 .select(df.case_id, df.case_opened_date, df.case_status, df.SLA_days)
 .where(df.case_status == 'Open')
 .sort(df.case_opened_date.desc())
 .show(1))



+----------+----------------+-----------+-----------+
|   case_id|case_opened_date|case_status|   SLA_days|
+----------+----------------+-----------+-----------+
|1013863552|    9/9/17 11:43|       Open|65.86542824|
+----------+----------------+-----------+-----------+
only showing top 1 row



How long has the oldest (in terms of days since opened) currently opened issue been open?


In [6]:
(df
 .select(df.case_id, df.case_opened_date, df.case_status, df.SLA_days)
 .where(df.case_status == 'Open')
 .sort(df.SLA_days.desc())
 .show(1))


+----------+----------------+-----------+--------+
|   case_id|case_opened_date|case_status|SLA_days|
+----------+----------------+-----------+--------+
|1014707122|   7/20/18 15:11|       Open|   999.0|
+----------+----------------+-----------+--------+
only showing top 1 row



How many Stray Animal cases are there?


In [7]:
(df
 .select(df.service_request_type)
 .where(df.service_request_type == 'Stray Animal')
 .agg(F.count(df.service_request_type))
 .show())

+---------------------------+
|count(service_request_type)|
+---------------------------+
|                      26760|
+---------------------------+



How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?


In [8]:
(df
 .select(df.service_request_type, df.dept_division)
 .where(df.dept_division == 'Field Operations')
 .where(df.service_request_type != 'Officer Standby')
 .agg(F.count(df.service_request_type))
 .show())

+---------------------------+
|count(service_request_type)|
+---------------------------+
|                     113902|
+---------------------------+



Create a new DataFrame without any information related to dates or location.


In [9]:
new_df = df.select(df.case_id,
                     df.case_late,
                     df.case_closed,
                     df.dept_division,
                     df.service_request_type,
                     df.case_status,
                     df.source_id)



Read dept.csv into a Spark DataFrame. Inspect the dept_name column. Replace the missing values with "other".


In [10]:
dept_df = spark.read.csv('dept.csv', header=True)
           
dept_df = dept_df.na.fill('other', ['dept_name'])

## Part 2


Convert the council_district column to a string column.


In [11]:
df.dtypes

[('case_id', 'string'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'string'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'string'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string')]

Extract the year from the case_closed_date column.


In [12]:
df = (df
 .withColumn('case_closed_date', F.to_timestamp(df.case_closed_date, 'M/d/y H:mm'))
 .withColumn('year', F.regexp_extract(F.col('case_closed_date'), r'\d+', 0)))

df.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 year                 | 2018                 
only showing top 1 row



Convert num_days_late from days to hours in new columns num_hours_late.


In [13]:
df = (df
 .withColumn('num_hours_late', df.num_days_late * 24))

df.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 year                 | 2018                 
 num_hours_late       | -23964.2102784       
only showing top 1 row



Convert the case_late column to a boolean column.


In [14]:
df = (df
 .withColumn('case_late',df.case_late == 'YES'))

df.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 year                 | 2018                 
 num_hours_late       | -23964.2102784       
only showing top 1 row



Convert the SLA_days columns to a double column.


In [15]:
df = (df
 .withColumn('SLA_days', F.col('SLA_days').cast('double')))

df.printSchema()

root
 |-- case_id: string (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: string (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- year: string (nullable = true)
 |-- num_hours_late: double (nullable = true)



Pull it all together


In [16]:
# Reassigned during exercises so it is already done.

## Part 3


Create a DataFrame with all combinations of council_district and service_request_type (regardless of whether the combination is observed in the data).


In [33]:
council_district = (df
 .select(df.council_district).distinct())

service_request_type  = (df
 .select(df.service_request_type ).distinct())

council_district.crossJoin(service_request_type).show()


+----------------+--------------------+
|council_district|service_request_type|
+----------------+--------------------+
|               7|Minimum Housing-O...|
|               7|        Tree Removal|
|               7| Service Information|
|               7|    Sign Maintenance|
|               7|Park Building Mai...|
|               7|Brush Property Da...|
|               7|Graffiti: Private...|
|               7|Traffic Sign Graf...|
|               7|License Renewal I...|
|               7|Used/Scrap Tire F...|
|               7|Guardrail- New Re...|
|               7|Markings Installa...|
|               7|CCO_Request for R...|
|               7|   Sewer Line Broken|
|               7|Zoning: Multi-Fam...|
|               7|Engineering Inves...|
|               7|    Zoning: Setbacks|
|               7|  Traffic Sign Faded|
|               7|     Permits, Fences|
|               7|Certificates of O...|
+----------------+--------------------+
only showing top 20 rows



Join the case data with the source and department data.


In [38]:
df.printSchema()

root
 |-- case_id: string (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: string (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- year: string (nullable = true)
 |-- num_hours_late: double (nullable = true)



In [39]:
dept_df.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = false)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [62]:
source_df = spark.read.csv('source.csv', header=True)

source_df.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [72]:
full_df = (df
 .join(dept_df, ['dept_division'], 'inner')
 .join(source_df, ['source_id'] , 'inner'))

full_df.show(vertical=True)


-RECORD 0--------------------------------------
 source_id              | svcCRMLS             
 dept_division          | Field Operations     
 case_id                | 1014127332           
 case_opened_date       | 1/1/18 0:42          
 case_closed_date       | 2018-01-01 12:29:00  
 SLA_due_date           | 9/26/20 0:42         
 case_late              | false                
 num_days_late          | -998.5087616000001   
 case_closed            | YES                  
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | Closed               
 request_address        | 2315  EL PASO ST,... 
 council_district       | 5                    
 year                   | 2018                 
 num_hours_late         | -23964.2102784       
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Services 
 dept_subject_to_SLA    | YES                  
 source_username        | svcCRMLS      

Are there any cases that do not have a request source?


In [146]:
full_df.where(full_df.source_username.isNull()).count()

0

In [84]:
full_df.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- case_id: string (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: string (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- year: string (nullable = true)
 |-- num_hours_late: double (nullable = true)
 |-- dept_name: string (nullable = false)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)
 |-- source_username: string (nullable = true)



## Part 4


Who are the top 10 service request types in terms of number of requests?


In [116]:
(full_df
 .groupBy(full_df.service_request_type)
 .agg(F.count(full_df.service_request_type).alias('service_count'))
 .sort(F.count(full_df.service_request_type).desc())
 .show(10, truncate=False))

+--------------------------------+-------------+
|service_request_type            |service_count|
+--------------------------------+-------------+
|No Pickup                       |89210        |
|Overgrown Yard/Trash            |66403        |
|Bandit Signs                    |32968        |
|Damaged Cart                    |31163        |
|Front Or Side Yard Parking      |28920        |
|Stray Animal                    |27361        |
|Aggressive Animal(Non-Critical) |25492        |
|Cart Exchange Request           |22608        |
|Junk Vehicle On Private Property|21649        |
|Pot Hole Repair                 |20827        |
+--------------------------------+-------------+
only showing top 10 rows



Who are the top 10 service request types in terms of average days late?


In [126]:
(full_df
    .select(full_df.service_request_type,full_df.num_days_late)
    .groupBy(full_df.service_request_type)
    .agg(F.round(F.avg(full_df.num_days_late),2).alias('average_days_late'))
    .sort(F.avg(full_df.num_days_late).desc())
    .show(10, truncate=False))

+--------------------------------------+-----------------+
|service_request_type                  |average_days_late|
+--------------------------------------+-----------------+
|Zoning: Junk Yards                    |175.96           |
|Labeling for Used Mattress            |162.43           |
|Record Keeping of Used Mattresses     |154.0            |
|Signage Requied for Sale of Used Mattr|151.64           |
|Storage of Used Mattress              |142.11           |
|Zoning: Recycle Yard                  |135.93           |
|Donation Container Enforcement        |131.76           |
|License Requied Used Mattress Sales   |128.8            |
|Traffic Signal Graffiti               |101.8            |
|Complaint                             |72.87            |
+--------------------------------------+-----------------+
only showing top 10 rows



Does number of days late depend on department?


In [128]:
(full_df
    .select(full_df.dept_name,full_df.num_days_late)
    .groupBy(full_df.dept_name)
    .agg(F.round(F.count(full_df.num_days_late),2).alias('average_days_late'))
    .sort(F.count(full_df.num_days_late).desc())
    .show(10, truncate=False))

+-------------------------+-----------------+
|dept_name                |average_days_late|
+-------------------------+-----------------+
|Code Enforcement Services|321984           |
|Solid Waste Management   |286287           |
|Animal Care Services     |119362           |
|Trans & Cap Improvements |97841            |
|Parks and Recreation     |19964            |
|Metro Health             |5313             |
|Customer Service         |2889             |
|Development Services     |1397             |
|other                    |198              |
|City Council             |0                |
+-------------------------+-----------------+



How do number of days late depend on department division and request type?


In [134]:
(full_df
    .groupBy(full_df.dept_name, full_df.service_request_type)
    .agg(F.round(F.count(full_df.num_days_late),2).alias('average_days_late'))
    .sort(F.count(full_df.num_days_late).desc())
    .show(10, truncate=False))

+-------------------------+--------------------------------+-----------------+
|dept_name                |service_request_type            |average_days_late|
+-------------------------+--------------------------------+-----------------+
|Solid Waste Management   |No Pickup                       |89210            |
|Code Enforcement Services|Overgrown Yard/Trash            |66403            |
|Code Enforcement Services|Bandit Signs                    |32968            |
|Solid Waste Management   |Damaged Cart                    |31163            |
|Code Enforcement Services|Front Or Side Yard Parking      |28920            |
|Animal Care Services     |Stray Animal                    |27361            |
|Animal Care Services     |Aggressive Animal(Non-Critical) |25492            |
|Solid Waste Management   |Cart Exchange Request           |22608            |
|Code Enforcement Services|Junk Vehicle On Private Property|21649            |
|Trans & Cap Improvements |Pot Hole Repair          

In [147]:
full_df.write.csv('full_311', header=True, mode='overwrite')
